# Analizar Mi Spotify
Se va realizar un análisis de mi cuenta de Spotify, donde se tratará de revisar:
1. Mis 25 artistas favoritos (más escuchados)
1. Las canciones más populares de mis artistas favoritos
1. Las canciones con mayor duración de mis artistas favoritos
1. La cantidad de albumes de mis artistas favoritos
1. La cantidad de singles de mis artistas favoritos
1. Los albumes más populares de mis artistas favoritos
1. Los albumes con mayor duración de mis artistas favoritos
1. Los albumes con mayor cantidad de canciones de mis artistas favoritos

## Pasos previos:
Para poder utilizar el API de spotify se necesita cumplir con ciertos requisitos previos los cuales se pueden encontrar en la documentación de la libreria de Spotipy: https://spotipy.readthedocs.io/en/2.24.0/#getting-started.
Una vez obtenido el client_id y el client_secret los almaceno en unos files (.client_id y .client_secret) para leerlos para el uso del proyecto

## Importaciones de librerias

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from datetime import datetime
import os
import time

In [2]:
# Eliminar el archivo de caché
if os.path.exists('.cache'):
    os.remove('.cache')

## Extraccion

### Extraer: Consultar los para la conexión del API
Obtener los parámetros para conectarse al API de Spotify

In [3]:
# Mi client ID obtenido previamente y guardado en un archivo .client_id
client_id = open('.client_id','r')
client_id = client_id.read()

In [4]:
# Mi client Secret obtenido previamente y guardado en un archivo .client_secret
client_secret = open('.client_secret','r')
client_secret = client_secret.read()

In [5]:
# Configuración de la autenticación
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri='http://localhost:3000',
    scope="user-top-read"
))

### Extraer: Mis artistas favoritos
Obtener un dataframe de los 50 artistas que más escucho

In [6]:
res_user_top_artists = sp.current_user_top_artists(limit=30, time_range='long_term')

In [7]:
list_user_top_artists_extract = []
for artist in enumerate(res_user_top_artists['items']):
    id_artist = artist[1]['id']
    name_artist = artist[1]['name']
    q_followers = artist[1]['followers']['total']
    popularity = artist[1]['popularity']
    list_user_top_artists_extract.append(
        [
            id_artist,
            name_artist,
            q_followers,
            popularity
        ]
    )

In [8]:
df_user_top_artists_extract = pd.DataFrame(list_user_top_artists_extract, columns = ['id_artist', 'name_artist', 'q_followers', 'popularity'])

In [9]:
df_user_top_artists_extract['extract_datetime'] = datetime.now()

In [10]:
df_user_top_artists_extract.to_csv('fuente/extraccion/user_top_artists.csv', sep='|', index=False)

In [11]:
df_user_top_artists_extract.to_excel('fuente/extraccion/user_top_artists.xlsx', index=False)

### Extraer: Los albumes de las canciones más populares de mis artistas favoritos
Obtener un dataframe de los albumes de las canciones más popualares de mis artistas favoritos

In [26]:
list_top_albums_fav_artists_extract = []
list_genres_albums_extract = []
dict_album = {}
# list_unique_id_albums = df_top_tracks_fav_artists_extract['id_album'].unique()
list_unique_id_albums_artists = df_top_tracks_fav_artists_extract[['id_album', 'id_artist']].drop_duplicates()
#for id_album in list_unique_id_albums:
for index, row in list_unique_id_albums_artists.iterrows():
    time.sleep(1)
    dict_album = sp.album(row['id_album'], 'US')
    total_tracks_album = dict_album['total_tracks']
    id_album = dict_album['id']
    name_album = dict_album['name']
    release_date_album = dict_album['release_date']
    type_album = dict_album['album_type']
    #id_artist = dict_album['artists'][0]['id']
    id_artist = row['id_artist']
    name_artist = dict_album['artists'][0]['name']
    duration_ms_total_album = 0
    populariy_album = dict_album['popularity']
    
    list_tracks_album = dict_album['tracks']['items']
    for track in list_tracks_album:
        duration_ms_total_album = duration_ms_total_album + int(track['duration_ms'])
    list_top_albums_fav_artists_extract.append(
        [
            id_album
            ,id_artist
            ,name_album
            ,type_album
            ,total_tracks_album
            ,release_date_album
            ,duration_ms_total_album
            ,populariy_album
        ]     
    )
    
print("CONSULTA EXITOSA")

CONSULTA EXITOSA


In [27]:
df_top_albums_fav_artists_extract = pd.DataFrame(list_top_albums_fav_artists_extract, columns = [
            'id_album'
            , 'id_artist'
            , 'name_album'
            , 'type_album'
            , 'total_tracks_album'
            , 'release_date_album'
            , 'duration_ms_total_album'
            , 'populariy_album'
])

In [28]:
df_top_albums_fav_artists_extract['extract_datetime'] = datetime.now()

In [29]:
df_top_albums_fav_artists_extract.to_csv('fuente/extraccion/top_albums_fav_artists.csv', sep='|', index=False)

In [30]:
df_top_albums_fav_artists_extract.to_excel('fuente/extraccion/top_albums_fav_artists.xlsx', index=False)

### Todos los Albumnes de mis artistas favoritos

In [12]:
list_tracks_album = []
list_total_albums_albums = []
#Iteración de artistas
for index, row in df_user_top_artists_extract.iterrows():
    print("///////////////////////////////////")
    print(row['name_artist'])
    id_artist = row['id_artist']
    limit = 50
    offset = 0
    country = 'US'
    include_groups= 'album'
    #Iteración de paginación de albumes del artista
    while True:
        results = sp.artist_albums(id_artist, country=country, include_groups=include_groups,limit=limit, offset=offset)
        #Iteración de los albumnes del artista
        for item in enumerate(results['items']):
            # Pausa para manejar límites de tasa
            time.sleep(1)
            item = item[1]
            type_album = item['album_type']
            total_tracks_album = item['total_tracks']
            id_album = item['id']
            name_album = item['name']
            release_date_album = item['release_date']
            print("\t" + name_album + "\t" + type_album)
            #Más informacion del album
            dict_album = sp.album(id_album, 'US')
            popularity_album = dict_album['popularity']
            duration_ms_total_album = 0            
            list_tracks_album = dict_album['tracks']['items']
            for track in list_tracks_album:
                duration_ms_total_album = duration_ms_total_album + int(track['duration_ms'])       
            list_total_albums_albums.append(
                [
                    id_album
                    , id_artist
                    , name_album
                    , type_album
                    , popularity_album
                    , total_tracks_album
                    , release_date_album
                    , duration_ms_total_album
                ]
            )
        if len(results['items']) < limit:
            break
        offset = offset + limit
        
        # Pausa para manejar límites de tasa
        time.sleep(1)
        
print("Consulta exitosa")

///////////////////////////////////
Gorillaz
	Cracker Island (Deluxe)	album
	Cracker Island	album
	Song Machine, Season One: Strange Timez (Deluxe)	album
	The Now Now	album
	Humanz (Deluxe)	album
	Demon Days Live at the Manchester Opera House	album
	Plastic Beach	album
	The Fall	album
	D-Sides [Special Edition]	album
	D-Sides	album
	Demon Days	album
	Laika Come Home	album
	Gorillaz	album
///////////////////////////////////
Proleter
	Fairuz	album
	Rookie (Remastered)	album
	Feeding The Lions (Remastered)	album
	Curses From Past Times (Remastered)	album
///////////////////////////////////
The Weeknd
	Starboy (Deluxe)	album
	Live At SoFi Stadium	album
	Dawn FM (Alternate World)	album
	Dawn FM	album
	After Hours (Deluxe)	album
	After Hours	album
	My Dear Melancholy,	album
	Starboy	album
	Beauty Behind The Madness	album
	Kiss Land (Deluxe)	album
	Kiss Land	album
	Trilogy	album
	Echoes Of Silence (Original)	album
	Thursday (Original)	album
	House Of Balloons (Original)	album
////////////////

	Reanimation	album
	Hybrid Theory (Bonus Edition)	album
///////////////////////////////////
VALORANT
	MV//MNT VOL. 02	album
	MV//MNT	album
///////////////////////////////////
Pendulum
	The Reworks	album
	Immersion	album
	Live at Brixton Academy	album
	In Silico	album
	Hold Your Colour (Deluxe)	album
	Hold Your Colour	album
///////////////////////////////////
Knife Party
	Abandon Ship	album
///////////////////////////////////
Bad Bunny
	nadie sabe lo que va a pasar mañana	album
	Un Verano Sin Ti	album
	EL ÚLTIMO TOUR DEL MUNDO	album
	LAS QUE NO IBAN A SALIR	album
	YHLQMDLG	album
	OASIS	album
	X 100PRE	album
///////////////////////////////////
Anuel AA
	LLNM2	album
	Las Leyendas Nunca Mueren	album
	Los Dioses	album
	Emmanuel	album
	Real Hasta la Muerte	album
///////////////////////////////////
M|O|O|N
	Come! See!!	album
	Demos 2014-2017	album
	Clinically Blasé	album
///////////////////////////////////
La Mente
	Millonarios del Alma	album
	PLAYLIZT Presenta: la Mente (En Vivo)	album
	Soni

In [13]:
list_tracks_album = []
list_total_singles_albums = []
#Iteración de artistas
for index, row in df_user_top_artists_extract.iterrows():
    print("///////////////////////////////////")
    print(row['name_artist'])
    id_artist = row['id_artist']
    limit = 50
    offset = 0
    country = 'US'
    include_groups= 'single'
    #Iteración de paginación de albumes del artista
    while True:
        results = sp.artist_albums(id_artist, country=country, include_groups=include_groups, limit=limit, offset=offset)
        #Iteración de los albumnes del artista
        for item in enumerate(results['items']):
            # Pausa para manejar límites de tasa
            time.sleep(1)
            item = item[1]
            type_album = item['album_type']
            total_tracks_album = item['total_tracks']
            id_album = item['id']
            name_album = item['name']
            release_date_album = item['release_date']
            print("\t" + name_album + "\t" + type_album)
            #Más informacion del album
            dict_album = sp.album(id_album, 'US')
            popularity_album = dict_album['popularity']
            duration_ms_total_album = 0            
            list_tracks_album = dict_album['tracks']['items']
            for track in list_tracks_album:
                duration_ms_total_album = duration_ms_total_album + int(track['duration_ms'])       
            list_total_singles_albums.append(
                [
                    id_album
                    , id_artist
                    , name_album
                    , type_album
                    , popularity_album
                    , total_tracks_album
                    , release_date_album
                    , duration_ms_total_album
                ]
            )
        if len(results['items']) < limit:
            break
        offset = offset + limit
        
        # Pausa para manejar límites de tasa
        time.sleep(1)
        
print("Consulta exitosa")

///////////////////////////////////
Gorillaz
	New Gold (feat. Tame Impala and Bootie Brown) [Dom Dolla Remix]	single
	Baby Queen	single
	New Gold (feat. Tame Impala and Bootie Brown)	single
	Cracker Island (feat. Thundercat)	single
	Meanwhile EP	single
	The Now Now (Gorillaz 20 Mix)	single
	Demon Days (Gorillaz 20 Mix)	single
	Humanz (Gorillaz 20 Mix)	single
	Gorillaz (Gorillaz 20 Mix)	single
	Song Machine Episode 8	single
	Song Machine, Season One: Strange Timez (Gorillaz 20 Mix)	single
	Mud’z Massive Machine Mix	single
	Russel Hobbs Presents a Flamin' Hot Song Machine Mix	single
	Song Machine Made by 2D From Gorillaz	single
	The Machine Is On!! A Song Machine Mix by Noodle	single
	Song Machine Episode 7	single
	Song Machine Episode 6	single
	Song Machine Episode 5	single
	Song Machine Episode 4	single
	How Far? (feat. Tony Allen and Skepta)	single
	Song Machine Episode 3	single
	Song Machine Episode 2	single
	Song Machine Episode 1	single
	Song Machine Theme Tune	single
	Tranz (Poté 

	Lunar Beats | Club 2 Arena Theme - Teamfight Tactics	single
	Run It	single
	Rell, the Iron Maiden	single
	Battle Queens - 2020	single
	Seraphine, the Starry-Eyed Songstress	single
	Take Over (ford. Remix)	single
	Samira, the Desert Rose	single
	Take Over	single
	2020 World Championship Theme	single
	PsyOps - 2020	single
	Yone, the Unforgotten	single
	Lillia, the Bashful Bloom	single
	Volibear, the Relentless Storm	single
	Fiddlesticks, the Harbinger of Doom	single
	Sett, the Boss	single
	Aphelios, the Weapon of the Faithful	single
	Senna, the Redeemer	single
	Warriors	single
	Start It Up	single
	Phoenix (1788-L Remix)	single
	Phoenix (Carpenter Brut Remix)	single
	Phoenix (Blanke Remix)	single
	Phoenix	single
	2019 World Championship Theme	single
	Bring Home The Glory	single
	Awaken	single
	Raise Up The Lights (2018 All-Star Event)	single
	RISE (Remix)	single
	RISE	single
	2018 World Championship Theme	single
	2017 World Championship Theme	single
	2018 Rift Rivals Theme	single
	2018 M

	What Kind Of Man (Nicolas Jaar Remix)	single
	Lover To Lover (Ceremonials Tour Version)	single
	What The Water Gave Me	single
	Shake It Out	single
	You've Got The Love	single
	Heavy In Your Arms	single
	You've Got The Love (Jamie xx Rework)	single
	A Lot of Love. a Lot of Blood	single
	Dog Days Are Over	single
	Kiss With A Fist	single
///////////////////////////////////
Metro Boomin
	pop ur shit	single
	née-nah	single
	just like me	single
	dangerous	single
	Creepin' (Remix)	single
	Intro I 2020 (Only 1 Remix)	single
	Blue Pill	single
	Perfect Timing (Intro)	single
	Call Me	single
	Chanel Vintage (feat. Future & Young Thug) - Single	single
	Chanel Vintage (feat. Future & Young Thug) - Single	single
///////////////////////////////////
El Cuarteto De Nos
	Chivo Expiatorio (En Vivo)	single
	El Cinturón Gris (En Vivo)	single
	Rorschach	single
	Maldito Show	single
	La Ciudad Sin Alma	single
	Fiesta en lo del Dr. Hermes (En Vivo)	single
	Fiesta en lo del Dr. Hermes	single
	Contrapunto para H

	We Made You (International Version)	single
	Shake That (Radio Edit Version)	single
	When I'm Gone	single
	Ass Like That	single
	Mockingbird	single
	Just Lose It	single
	Infinite	single
///////////////////////////////////
K/DA
	ALL OUT	single
	MORE	single
	THE BADDEST	single
	POP/STARS	single
///////////////////////////////////
Die Antwoord
	Everything is Perfect	single
	REANIMATED	single
	BARUCH IN JOU OEG	single
	Age Of Illusion	single
	ZEF GOEMA MEGAMIX	single
	Baita Jou Sabela	single
	DntTakeMe4aPoes	single
	2•GOLDEN DAWN•7	single
	Love Drug	single
	Fat Faded Fuck Face	single
	We Have Candy	single
	Banana Brain	single
	Pitbull Terrier	single
	5 - EP	single
///////////////////////////////////
Red Hot Chili Peppers
	The Shape I'm Takin'	single
	Eddie	single
	Tippa My Tongue	single
	Nerve Flip	single
	Not the One	single
	Poster Child	single
	Black Summer	single
	Goodbye Angels	single
	We Turn Red	single
	The Getaway	single
	Dark Necessities	single
	This Is the Kitt / Brave from Afar	s

	WYA REMIX BLACK AND YELLOW (feat. iZaak & Pirlo)	single
	Hasta la Muerte	single
	XQCP	single
	Bellakita	single
	Toki	single
	Kilerito	single
	No Te Quieren Conmigo (Remix) (Gaby Music, Anuel AA, Luar LA L)	single
	BBY BOO (REMIX)	single
	VVS Switch (feat. Hades66, Luar La L & CDobleta) [Remix]	single
	Tacos Gucci	single
	Rebound (feat. Anuel AA)	single
	Luces Tenues	single
	Arcangel Es Chota	single
	Glock, Glock, Glock	single
	Alcohol	single
	OA	single
	Oh Na Na	single
	Corazón Roto pt. 3	single
	PARA SIEMPRE	single
	Podemos Repetirlo (Remix)	single
	PACTO (Remix) [feat. Bryant Myers & Dei V]	single
	Baby	single
	Milloneta	single
	Mejor Que Yo	single
	MI EXXX	single
	Triste Verano	single
	Diablo, Qué Chimba	single
	Mas Rica Que Ayer	single
	Drippin	single
	Wakanda	single
	Vibra	single
	Sufro	single
	Mientes	single
	Del Kilo	single
	Airbnb	single
	1ro	single
	La Máquina	single
	Hoodie	single
	Si Yo Me Muero	single
	Presidentes Muertos	single
	MUEVELO	single
	DURO	single
	Diamantes En M

	I Feel Good (Sak Noel X Salvi X Franklin Dam Remix)	single
	I Feel Good (R3HAB Remix)	single
	I Know You Want Me (Calle Ocho) [Helion Remix]	single
	I Feel Good	single
	Where the Country Girls At	single
	Ten Cuidado	single
	Cosita Linda	single
	Give It To Me	single
	Que Rica (Tocame)	single
	Voodoo	single
	Moviéndolo (Remix)	single
	Borracha (Pero Buena Muchacha)	single
	Backpack	single
	Te Quiero Baby (I Love You Baby)	single
	Mala (feat. Becky G & De La Ghetto)	single
	I Believe That We Will Win (World Anthem) [Thombs Remixes]	single
	I Believe That We Will Win (World Anthem)	single
	Suda (Remixes)	single
	Suda (Deluxe)	single
	Get Ready	single
	Suda	single
	Me Quedaré Contigo (Shndō Remix)	single
	Further Up (Na, Na, Na, Na, Na)	single
	Imagínate	single
	3 to Tango	single
	No Lo Trates	single
	YAYO	single
	La Reina De Blanco	single
	Body On My	single
	Dame Tu Cosita (feat. Cutty Ranks) [Remixes]	single
	Quiero Saber	single
	Dame Tu Cosita (feat. Cutty Ranks) [Radio Version]	single


In [14]:
list_tracks_album = []
list_total_compilations_albums = []
#Iteración de artistas
for index, row in df_user_top_artists_extract.iterrows():
    print("///////////////////////////////////")
    print(row['name_artist'])
    id_artist = row['id_artist']
    limit = 50
    offset = 0
    country = 'US'
    include_groups= 'compilation'
    #Iteración de paginación de albumes del artista
    while True:
        results = sp.artist_albums(id_artist, country=country, include_groups=include_groups, limit=limit, offset=offset)
        #Iteración de los albumnes del artista
        for item in enumerate(results['items']):
            # Pausa para manejar límites de tasa
            time.sleep(1)
            item = item[1]
            type_album = item['album_type']
            total_tracks_album = item['total_tracks']
            id_album = item['id']
            name_album = item['name']
            release_date_album = item['release_date']
            print("\t" + name_album + "\t" + type_album)
            #Más informacion del album
            dict_album = sp.album(id_album, 'US')
            popularity_album = dict_album['popularity']
            duration_ms_total_album = 0            
            list_tracks_album = dict_album['tracks']['items']
            for track in list_tracks_album:
                duration_ms_total_album = duration_ms_total_album + int(track['duration_ms'])       
            list_total_compilations_albums.append(
                [
                    id_album
                    , id_artist
                    , name_album
                    , type_album
                    , popularity_album
                    , total_tracks_album
                    , release_date_album
                    , duration_ms_total_album
                ]
            )
        if len(results['items']) < limit:
            break
        offset = offset + limit
        
        # Pausa para manejar límites de tasa
        time.sleep(1)
        
print("Consulta exitosa")

///////////////////////////////////
Gorillaz
	Gorillaz Are Ten - Spotify Radio Show 3	compilation
	Gorillaz Are Ten - Spotify Radio Show 2	compilation
	The Singles Collection 2001-2011	compilation
	Gorillaz Are Ten - Spotify Radio Show 1	compilation
	G-Sides	compilation
///////////////////////////////////
Proleter
///////////////////////////////////
The Weeknd
	The Highlights (Deluxe)	compilation
	The Highlights	compilation
///////////////////////////////////
League of Legends
///////////////////////////////////
Skrillex
///////////////////////////////////
Daft Punk
	Musique, Vol. 1	compilation
///////////////////////////////////
Nirvana
	Sliver - The Best Of The Box	compilation
	With The Lights Out - Box Set	compilation
	Nirvana	compilation
///////////////////////////////////
De Hofnar
	Armada Invites (In The Mix): De Hofnar	compilation
///////////////////////////////////
Florence + The Machine
	Under Heaven Over Hell	compilation
	Harder Than Hell	compilation
	My Favorite Ghosts	compi

In [ ]:
list_tracks_album = []
list_total_appears_on_albums = []
#Iteración de artistas
for index, row in df_user_top_artists_extract.iterrows():
    print("///////////////////////////////////")
    print(row['name_artist'])
    id_artist = row['id_artist']
    limit = 50
    offset = 0
    country = 'US'
    include_groups= 'appears_on'
    #Iteración de paginación de albumes del artista
    while True:
        results = sp.artist_albums(id_artist, country=country, include_groups=include_groups, limit=limit, offset=offset)
        #Iteración de los albumnes del artista
        for item in enumerate(results['items']):
            # Pausa para manejar límites de tasa
            time.sleep(1)
            item = item[1]
            type_album = item['album_type']
            total_tracks_album = item['total_tracks']
            id_album = item['id']
            name_album = item['name']
            release_date_album = item['release_date']
            print("\t" + name_album + "\t" + type_album)
            #Más informacion del album
            dict_album = sp.album(id_album, 'US')
            popularity_album = dict_album['popularity']
            duration_ms_total_album = 0            
            list_tracks_album = dict_album['tracks']['items']
            for track in list_tracks_album:
                duration_ms_total_album = duration_ms_total_album + int(track['duration_ms'])       
            list_total_appears_on_albums.append(
                [
                    id_album
                    , id_artist
                    , name_album
                    , type_album
                    , popularity_album
                    , total_tracks_album
                    , release_date_album
                    , duration_ms_total_album
                ]
            )
        if len(results['items']) < limit:
            break
        offset = offset + limit
        
        # Pausa para manejar límites de tasa
        time.sleep(1)
        
print("Consulta exitosa")

In [15]:
list_total_albums = list_total_albums_albums + list_total_singles_albums + list_total_compilations_albums ##+ list_total_appears_on_albums

In [31]:
df_albums_fav_artists_extract = pd.DataFrame(list_total_albums, columns = [
    'id_album'
    , 'id_artist'
    , 'name_album'
    , 'type_album'
    , 'popularity_album'
    , 'total_tracks_album'
    , 'release_date_album'
    , 'duration_ms_total_album'
    ])

In [32]:
df_albums_fav_artists_extract['extract_datetime'] = datetime.now()

In [33]:
df_albums_fav_artists_extract.to_csv('fuente/extraccion/albums_fav_artists.csv', sep='|', index=False)

In [34]:
df_albums_fav_artists_extract.to_excel('fuente/extraccion/albums_fav_artists.xlsx', index=False)

### Extraer: Las canciones más populares de mis artistas favoritos
Obtener un dataframe de las canciones más populares de mis artistas favoritos

In [35]:
list_dict_top_tracks_fav_artists_extract = []
for index, row in df_user_top_artists_extract.iterrows():
    index_artist = index
    name_artist = row['name_artist']
    id_artist = row['id_artist']
    list_dict_top_tracks_fav_artists_extract.append(
        dict(name_artist = name_artist, id_artist = id_artist, index_artist = index, top_tracks = sp.artist_top_tracks(id_artist, country='US'))
    )

In [36]:
list_top_tracks_fav_artists_extract = []
for row in list_dict_top_tracks_fav_artists_extract:
    index_track = 0
    for track in row['top_tracks']['tracks']:
        position_track = index_track
        user_rank_artist = row['index_artist']
        id_artist = row['id_artist']
        id_track = track['id']
        id_album = track['album']['id']
        name_track = track['name']
        duration_track_ms = track['duration_ms']
        popularity_track = track['popularity']
        number_track = track['track_number']
        list_top_tracks_fav_artists_extract.append(
            [
            id_track
            , id_artist
            , id_album
            , name_track
            , duration_track_ms
            , popularity_track
            , number_track
            , position_track
            ]
        )
        index_track = index_track + 1

In [37]:
df_top_tracks_fav_artists_extract = pd.DataFrame(list_top_tracks_fav_artists_extract, columns = [
            'id_track'
            , 'id_artist'
            , 'id_album'
            , 'name_track'
            , 'duration_track_ms'
            , 'popularity_track'
            , 'number_track'
            , 'position_track'])

In [38]:
df_top_tracks_fav_artists_extract['extract_datetime'] = datetime.now()

In [39]:
df_top_tracks_fav_artists_extract.to_csv('fuente/extraccion/top_tracks_fav_artists.csv', sep='|', index=False)

In [40]:
df_top_tracks_fav_artists_extract.to_excel('fuente/extraccion/top_tracks_fav_artists.xlsx', index=False)

## Transformacion

### Transformacion: Crear columna duration_track_ms en segundos y en minutos del df_top_tracks_fav_artists

In [41]:
df_top_tracks_fav_artists_transform = df_top_tracks_fav_artists_extract.copy()

In [42]:
df_top_tracks_fav_artists_transform = df_top_tracks_fav_artists_transform[
    ['id_track'
    , 'id_artist'
     , 'id_album'
     , 'name_track'
     , 'duration_track_ms'
     , 'popularity_track'
     , 'number_track'
     , 'position_track']
]

In [43]:
df_top_tracks_fav_artists_transform['duration_track_seconds'] = df_top_tracks_fav_artists_transform['duration_track_ms'] / 1000.0

In [44]:
df_top_tracks_fav_artists_transform['duration_track_minutes'] = df_top_tracks_fav_artists_transform['duration_track_ms'] / (1000.0 * 60.0)

In [45]:
df_top_tracks_fav_artists_transform['transform_datetime'] = datetime.now()

In [46]:
df_top_tracks_fav_artists_transform.head(5)

,id_track,id_artist,id_album,name_track,duration_track_ms,popularity_track,number_track,position_track,duration_track_seconds,duration_track_minutes,transform_datetime
0,0d28khcov6AiegSCpG5TuT,3AA28KZvwAUcZuOKwyblJQ,0bUTHlWbkSQysoM3VsWldT,Feel Good Inc.,222640,84,6,0,222.640,3.710667,2024-07-16 08:36:45.339420
1,0q6LuUqGLUiCPP1cbdwFs3,3AA28KZvwAUcZuOKwyblJQ,2dIGnmEIy1WZIcZCFSj6i8,On Melancholy Hill,233866,78,10,1,233.866,3.897767,2024-07-16 08:36:45.339420
2,1RKUoGiLEbcXN4GY4spQDx,3AA28KZvwAUcZuOKwyblJQ,4tUxQkrduOE8sfgwJ5BI2F,Clint Eastwood,340920,75,5,2,340.920,5.682000,2024-07-16 08:36:45.339420
3,4Hff1IjRbLGeLgFgxvHflk,3AA28KZvwAUcZuOKwyblJQ,0bUTHlWbkSQysoM3VsWldT,DARE,244999,76,12,3,244.999,4.083317,2024-07-16 08:36:45.339420
4,1foMv2HQwfQ2vntFf9HFeG,3AA28KZvwAUcZuOKwyblJQ,2dIGnmEIy1WZIcZCFSj6i8,Rhinestone Eyes,200173,75,4,4,200.173,3.336217,2024-07-16 08:36:45.339420


### Transformacion: Crear columna duration_ms_total_album en segundos y en minutos del df_top_albums_fav_artists

In [47]:
df_top_albums_fav_artists_transform = df_top_albums_fav_artists_extract.copy()

In [48]:
df_top_albums_fav_artists_transform = df_top_albums_fav_artists_transform[
    ['id_album'
     , 'id_artist'
     , 'name_album'
     , 'type_album'
     , 'total_tracks_album'
     , 'release_date_album'
     , 'duration_ms_total_album'
    ,  'populariy_album']
]

In [49]:
df_top_albums_fav_artists_transform['duration_total_seconds_album'] = df_top_albums_fav_artists_transform['duration_ms_total_album'] / 1000.0

In [50]:
df_top_albums_fav_artists_transform['duration_total_minutes_album'] = df_top_albums_fav_artists_transform['duration_ms_total_album'] / (1000.0 * 60.0)

In [51]:
df_top_albums_fav_artists_transform['transform_datetime'] = datetime.now()

In [52]:
df_top_albums_fav_artists_transform.head(5)

,id_album,id_artist,name_album,type_album,total_tracks_album,release_date_album,duration_ms_total_album,populariy_album,duration_total_seconds_album,duration_total_minutes_album,transform_datetime
0,0bUTHlWbkSQysoM3VsWldT,3AA28KZvwAUcZuOKwyblJQ,Demon Days,album,15,2005-05-23,3102755,80,3102.755,51.712583,2024-07-16 08:36:48.648229
1,2dIGnmEIy1WZIcZCFSj6i8,3AA28KZvwAUcZuOKwyblJQ,Plastic Beach,album,16,2010-03-03,3413486,76,3413.486,56.891433,2024-07-16 08:36:48.648229
2,4tUxQkrduOE8sfgwJ5BI2F,3AA28KZvwAUcZuOKwyblJQ,Gorillaz,album,17,2001-03-26,3880901,70,3880.901,64.681683,2024-07-16 08:36:48.648229
3,0NvirtaDCaZU5PAW1O5FDE,3AA28KZvwAUcZuOKwyblJQ,Humanz (Deluxe),album,26,2017-04-28,4151469,70,4151.469,69.191150,2024-07-16 08:36:48.648229
4,4wtZQMNTC1O79kDxMBsEan,3AA28KZvwAUcZuOKwyblJQ,Cracker Island,album,10,2023-02-24,2249194,67,2249.194,37.486567,2024-07-16 08:36:48.648229


### Transformacion: Crear columna duration_ms_total_album en segundos y en minutos del df_albums_fav_artists_extract

In [53]:
df_albums_fav_artists_transform = df_albums_fav_artists_extract.copy()

In [54]:
df_albums_fav_artists_transform = df_albums_fav_artists_transform[
    ['id_album'
     , 'id_artist'
     , 'name_album'
     , 'type_album'
     , 'popularity_album'
     , 'total_tracks_album'
     , 'release_date_album'
     , 'duration_ms_total_album']
]

In [55]:
df_albums_fav_artists_transform['duration_seconds_total_album'] = df_albums_fav_artists_transform['duration_ms_total_album'] / 1000.0

In [56]:
df_albums_fav_artists_transform['duration_minutes_total_album'] = df_albums_fav_artists_transform['duration_ms_total_album'] / (1000.0 * 60.0)

### Transformacion: Formatear campo release_date_album df_albums_fav_artists_extract

In [57]:
df_albums_fav_artists_transform['release_date_album'] = df_albums_fav_artists_transform['release_date_album'].apply(
    lambda x: x + '-01-01' if pd.Series(x).str.match(r'^\d{4}$')[0] else x
)

In [58]:
df_albums_fav_artists_transform['transform_datetime'] = datetime.now()

### Transformacion: user_top_artists

In [59]:
df_user_top_artists_transform = df_user_top_artists_extract.copy()

In [60]:
df_user_top_artists_transform = df_user_top_artists_transform[
    ['id_artist'
     , 'name_artist'
     , 'q_followers'
     , 'popularity']
]

In [61]:
current_datetime = datetime.now()

In [62]:
df_user_top_artists_transform['transform_datetime'] = current_datetime

In [63]:
df_user_top_artists_transform.head(5)

,id_artist,name_artist,q_followers,popularity,transform_datetime
0,3AA28KZvwAUcZuOKwyblJQ,Gorillaz,11827037,79,2024-07-16 08:36:55.531502
1,5dCpFeKxLbycrnsjWZjha8,Proleter,119145,44,2024-07-16 08:36:55.531502
2,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,86127221,93,2024-07-16 08:36:55.531502
3,47mIJdHORyRerp4os813jD,League of Legends,1863031,75,2024-07-16 08:36:55.531502
4,5he5w2lnU9x7JFhnwcekXX,Skrillex,7604981,75,2024-07-16 08:36:55.531502


### Exportación final

In [64]:
df_albums_fav_artists_transform.to_csv('fuente/transformacion/albums_fav_artists.csv', sep='|', index=False)

In [65]:
df_albums_fav_artists_transform.to_excel('fuente/transformacion/albums_fav_artists.xlsx', index=False)

In [66]:
df_top_tracks_fav_artists_transform.to_csv('fuente/transformacion/top_tracks_fav_artists.csv', sep='|', index=False)

In [67]:
df_top_tracks_fav_artists_transform.to_excel('fuente/transformacion/top_tracks_fav_artists.xlsx', index=False)

In [68]:
df_user_top_artists_transform.to_csv('fuente/transformacion/user_top_artists.csv', sep='|', index=False)

In [69]:
df_user_top_artists_transform.to_excel('fuente/transformacion/user_top_artists.xlsx', index=False)

In [70]:
df_top_albums_fav_artists_transform.to_csv('fuente/transformacion/top_albums_fav_artists.csv', sep='|', index=False)

In [71]:
df_top_albums_fav_artists_transform.to_excel('fuente/transformacion/top_albums_fav_artists.xlsx', index=False)